**SVM non-continuous var**

genetic opt  noncontin features (SMOTE) and imputed only
1_2_1c

In [40]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn_genetic import *
from sklearn_genetic.space import *
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix, roc_auc_score
from collections import Counter
from imblearn.over_sampling import SMOTE, SMOTEN

'''
param_grid = {
    "clf__n_estimators": Integer(50, 600),
    "clf__loss": Categorical(["absolute_error", "squared_error"]),
    "clf__max_depth": Integer(2, 10),
    "clf__learning_rate": Continuous(0.001, 0.01, distribution="log-uniform")}
    
gaSearch = GASearchCV(estimator=clf, cv=3, 
            param_grid=param_grid, scoring='accuracy', 
            population_size=50, generations=80, 
            crossover_probability=0.2, mutation_probability=0.8, 
            tournament_size=3, elitism=True, 
            verbose=True, keep_top_k=1, 
            criteria='max', algorithm='eaMuPlusLambda', 
            refit=True, n_jobs=1, 
            pre_dispatch='4*n_jobs', 
            error_score=np.nan, return_train_score=False, 
            log_config=None)

gaSearch.fit(X_train, y_train)
y_predict_ga = gaSearch.predict(X_test)
mse = mean_squared_error(y_test, y_predict_ga)

'''

#'''
pd.set_option("display.max_columns", None)

print('SMOTEN non-contin')
print()

estimate = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, 
                         fit_intercept=True, max_iter=1000, 
                         tol=0.001, shuffle=True, verbose=0, epsilon=0.1, 
                         n_jobs=2, random_state=None, learning_rate='optimal', 
                         eta0=0.0, power_t=0.5, early_stopping=True, 
                         validation_fraction=0.1, n_iter_no_change=5, 
                         class_weight=None, warm_start=False, average=False)
model = Pipeline([('scaler',  StandardScaler()), ('estimate', estimate)])


#cv = KFold(n_splits=5, shuffle=True)

alpha_lst = [i/10000 for i in range(1,10000,1000)]
fraction_lst = [i/10 for i in range(1,10)]

param_grid = [{
    'estimate__alpha': alpha_lst,
    'estimate__validation_fraction': fraction_lst}]


#k = 5
#kf = KFold(n_splits=k, shuffle=True, random_state=None)

#smoten_noncontin = pd.read_csv('smoten_noncontin.csv', index_col=False)
#X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 0) #train split

smoten_noncontin = pd.read_csv('smoten_noncontin.csv', index_col=False)

X = smoten_noncontin.drop('outcome',axis= 1)
y = smoten_noncontin['outcome']

#acc_score = []
#for train_index , test_index in kf.split(X):
#    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
#    y_train , y_test = y[train_index] , y[test_index]
    
#    model.fit(X_train, y_train)
#    y_pred = model.predict(X_test)
     
#    acc = accuracy_score(y_pred , y_test)
#    acc_score.append(acc)
     
#'''
gsSearch = GridSearchCV(estimator=model, param_grid=param_grid, 
             scoring=['f1_weighted'], n_jobs=2, 
             refit='f1_weighted', cv=5, 
             verbose=0, pre_dispatch='4*n_jobs', 
             error_score=np.nan, return_train_score=False)

gsSearch.fit(X_train, y_train)

print('best_estimator = ', gsSearch.best_estimator_)
best_grid = gsSearch.best_estimator_

print('best_params = ', gsSearch.best_params_)
print('best_score = ', gsSearch.best_score_)


y_pred=best_grid.predict(X_test)
print('confusion_matrix \n', confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print('classification_report \n', classification_report(y_test,y_pred))


SMOTEN non-contin

best_estimator =  Pipeline(steps=[('scaler', StandardScaler()),
                ('estimate',
                 SGDClassifier(alpha=0.2001, early_stopping=True, n_jobs=2,
                               validation_fraction=0.9))])
best_params =  {'estimate__alpha': 0.2001, 'estimate__validation_fraction': 0.9}
best_score =  0.6937508988706119
confusion_matrix 
 [[5718 1224]
 [2940 4182]]
Accuracy Score 0.7039249146757679
classification_report 
               precision    recall  f1-score   support

           0       0.66      0.82      0.73      6942
           1       0.77      0.59      0.67      7122

    accuracy                           0.70     14064
   macro avg       0.72      0.71      0.70     14064
weighted avg       0.72      0.70      0.70     14064

